# 이전 대화를 기억하는 Chain 생성방법

이 내용을 이해하기 위한 사전 지식
- `RunnableWithMessageHistory`: [https://wikidocs.net/235581](https://wikidocs.net/235581)

In [1]:
# API KEY를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API KEY 정보로드
load_dotenv()

True

In [2]:
# LangSmith 추적을 설정합니다. https://smith.langchain.com
# !pip install langchain-teddynote
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("CH12-RAG")

LangSmith 추적을 시작합니다.
[프로젝트명]
CH12-RAG


## 1. 일반 Chain 에 대화기록 추가

In [7]:
import os

In [8]:
from langchain_upstage import ChatUpstage, UpstageEmbeddings

In [9]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser


# 프롬프트 정의
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "당신은 병원 AI 지원 시스템입니다. 보호자 또는 환자가 의료 상태에 대해 질문할 때, 환자의 최신 의료 기록을 바탕으로 적절한 답변을 제공해주세요.",
        ),
        # 대화기록용 key 인 chat_history 는 가급적 변경 없이 사용하세요!
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "#Question:\n{question}"),  # 사용자 입력을 변수로 사용
    ]
)

# llm 생성
llm = ChatUpstage(temperature=0, api_key=os.getenv("UPSTAGE_API_KEY"))

# 일반 Chain 생성
chain = prompt | llm | StrOutputParser()

대화를 기록하는 체인 생성(`chain_with_history`)

In [16]:
# 세션 기록을 저장할 딕셔너리
store = {}


# 세션 ID를 기반으로 세션 기록을 가져오는 함수
def get_session_history(session_ids):
    print(f"[대화 세션ID]: {session_ids}")
    if session_ids not in store:  # 세션 ID가 store에 없는 경우
        # 새로운 ChatMessageHistory 객체를 생성하여 store에 저장
        store[session_ids] = ChatMessageHistory()
    return store[session_ids]  # 해당 세션 ID에 대한 세션 기록 반환


chain_with_history = RunnableWithMessageHistory(
    chain,
    get_session_history,  # 세션 기록을 가져오는 함수
    input_messages_key="question",  # 사용자의 질문이 템플릿 변수에 들어갈 key
    history_messages_key="chat_history",  # 기록 메시지의 키
)

첫 번째 질문 실행

In [17]:
chain_with_history.invoke(
    # 질문 입력
    {"question": "제 이름은 김성연입니다"},
    # 세션 ID 기준으로 대화를 기록합니다.
    config={"configurable": {"session_id": "남A"}},
)

[대화 세션ID]: 남A


'안녕하세요, 김성연님. 저는 AI 지원 시스템입니다. 어떤 의료 관련 질문이 있으신가요?'

이어서 질문 실행

In [24]:
chain_with_history.invoke(
    # 질문 입력
    {"question": "내 이름을 알려줘"},
    # 세션 ID 기준으로 대화를 기록합니다.
    config={"configurable": {"session_id": patient_id}},
)

[대화 세션ID]: 남A


'죄송합니다, 김성연님. 제가 이해하지 못한 것 같습니다. 다른 질문이 있으시면 도와드릴 수 있을까요?'

## 2. RAG + RunnableWithMessageHistory

먼저 일반 RAG Chain 을 생성합니다. 단, 6단계의 prompt 에 `{chat_history}` 를 꼭 추가합니다.

In [62]:
patient_id = "여A"

In [63]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PDFPlumberLoader
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from operator import itemgetter
from langchain.vectorstores import Chroma


patient_id = "여A"

# Embeddings setup
embeddings = UpstageEmbeddings(
  api_key=os.getenv("UPSTAGE_API_KEY"),
  model="solar-embedding-1-large"
)

vectordb = Chroma(
    persist_directory=f'.cache/db/{patient_id}',
    embedding_function=embeddings
)


retriever = vectordb.as_retriever()

# 단계 6: 프롬프트 생성(Create Prompt)

template ="""You are a compassionate, articulate physician.
Your goal is to explain medical information in a way that is easy for your patients to understand, avoiding complex medical jargon as much as possible.
Given a medical document, it's your job to explain the key information that the patient or their family asks about in a patient-friendly format. 
When specific details are provided, such as diagnosis codes or medical history, simplify these terms and explain them in a way that is easy to understand.

If you don't know the answer, just say that you don't know.

Don't say what is in the history.

Answer in Korean.

#Previous Chat History:
{chat_history}

#Question: 
{question} 

#Document:
{context}


#Answer:"""


prompt = PromptTemplate.from_template(template)

# 단계 7: 언어모델(LLM) 생성
# 모델(LLM) 을 생성합니다.
llm = ChatUpstage(api_key=os.getenv("UPSTAGE_API_KEY"))

# 단계 8: 체인(Chain) 생성
chain = (
    {
        "context": itemgetter("question") | retriever,
        "question": itemgetter("question"),
        "chat_history": itemgetter("chat_history"),
    }
    | prompt
    | llm
    | StrOutputParser()
)

대화를 저장할 함수 정의

In [69]:
# 세션 기록을 저장할 딕셔너리
store = {}


# 세션 ID를 기반으로 세션 기록을 가져오는 함수
def get_session_history(session_ids):
    print(f"[대화 세션ID]: {session_ids}")
    if session_ids not in store:  # 세션 ID가 store에 없는 경우
        # 새로운 ChatMessageHistory 객체를 생성하여 store에 저장
        store[session_ids] = ChatMessageHistory()
    return store[session_ids]  # 해당 세션 ID에 대한 세션 기록 반환


# 대화를 기록하는 RAG 체인 생성
rag_with_history = RunnableWithMessageHistory(
    chain,
    get_session_history,  # 세션 기록을 가져오는 함수
    input_messages_key="question",  # 사용자의 질문이 템플릿 변수에 들어갈 key
    history_messages_key="chat_history",  # 기록 메시지의 키
)

In [70]:
store

{}

첫 번째 질문 실행

In [71]:
rag_with_history.invoke(
    # 질문 입력
    {"question": "어머니 상태는 어떠신가요?"},
    # 세션 ID 기준으로 대화를 기록합니다.
    config={"configurable": {"session_id": "rag123"}},
)

[대화 세션ID]: rag123


'어머니의 상태는 현재 의료진의 관찰과 치료를 받고 계십니다. 어머니는 91세의 여성으로, 516호에 입원 중이십니다. 어머니는 대소변 조절이 어려우셔서 기저귀를 착용하고 계시며, 본원 에어매트를 사용하고 계십니다. 어머니는 정서적인 어려움을 겪고 계시며, "이렇게 살아서 뭐하나, 죽고 싶다"라는 표현을 하셨습니다. 또한, 어머니는 공격적인 언어를 사용하며, 옆에 있는 환자와의 대화를 혼동하는 경우가 있습니다. 의료진은 어머니의 상태를 지속적으로 관찰하고, 필요한 조치를 취하고 있습니다.'

이어진 질문 실행

In [72]:
rag_with_history.invoke(
    # 질문 입력
    {"question": "말한거 말고 다른 문제는 없으신가요?"},
    # 세션 ID 기준으로 대화를 기록합니다.
    config={"configurable": {"session_id": "rag123"}},
)

[대화 세션ID]: rag123


'어머니의 건강 상태에 대해 더 자세히 알고 싶으신 내용이 있나요?'

In [73]:
rag_with_history.invoke(
    # 질문 입력
    {"question": "약은 잘 드시는지 궁금합니다."},
    # 세션 ID 기준으로 대화를 기록합니다.
    config={"configurable": {"session_id": "rag123"}},
)

[대화 세션ID]: rag123


'어머니께서는 약을 잘 드시고 계십니다. 어머니께서는 아침과 저녁에 레바미피드와 마그밀정을 복용하고 계시며, 아침과 저녁에 아니스펜8시간이알서방정, 아세틸캡슐, 인데놀정10mg, 하이페질정5mg, 텔미로탄정을 복용하고 계십니다. 어머니께서는 아침, 점심, 저녁에 마그밀정을 복용하고 계십니다. 어머니께서는 아침에 마그밀정500mg, 아니스펜8시간이알서방정, 아세틸캡슐, 인데놀정10mg, 하이페질정5mg, 텔미로탄정을 복용하고 계십니다. 어머니께서는 저녁에 마그밀정500mg, 아니스펜8시간이알서방정, 아세틸캡슐, 인데놀정10mg, 하이페질정5mg, 텔미로탄정을 복용하고 계십니다. 어머니께서는 아침, 점심, 저녁에 일반식을 드시고 계십니다.'

In [60]:
store


{'rag123': InMemoryChatMessageHistory(messages=[HumanMessage(content='아버지 상태는 어떠신가요?'), AIMessage(content='아버님의 상태는 현재 몸이 약하시고, 스스로 움직이기 어려워서 다른 사람들의 도움이 필요합니다. 옷 입기, 세수하기, 양치하기, 목욕하기, 식사하기, 체위 변경하기, 일어나 앉기, 이동하기, 화장실 사용 등에 도움이 필요합니다. 또한, 의자나 휠체어로 이동하고, G-weakness로 인해 도움이 필요한 상태입니다. 의료진들은 아버님의 상태를 관찰하고, 필요한 조치를 취하고 있습니다.'), HumanMessage(content='말한거 말고 다른 문제는 없으신가요?'), AIMessage(content='아버님의 상태는 현재 몸이 약하시고, 스스로 움직이기 어려워서 다른 사람들의 도움이 필요합니다. 옷 입기, 세수하기, 양치하기, 목욕하기, 식사하기, 체위 변경하기, 일어나 앉기, 이동하기, 화장실 사용 등에 도움이 필요합니다. 또한, 의자나 휠체어로 이동하고, G-weakness로 인해 도움이 필요한 상태입니다. 의료진들은 아버님의 상태를 관찰하고, 필요한 조치를 취하고 있습니다.'), HumanMessage(content='지금 대화를 요약해주세요'), AIMessage(content='아버지의 건강 상태에 대해 대화를 나누었던 것 같습니다. 아버지는 몸이 약해져 스스로 움직이기 어려운 상태이며, 옷 입기, 세수하기, 양치하기, 목욕하기, 식사하기, 체위 변경하기, 일어나 앉기, 이동하기, 화장실 사용 등에 도움이 필요합니다. 또한, 의자나 휠체어로 이동하며, G-weakness로 인해 도움이 필요한 상태입니다. 의료진들은 아버지의 상태를 관찰하고 적절한 조치를 취하고 있습니다.')])}

In [61]:
rag_with_history.invoke(
    # 질문 입력
    {"question": "요약해줘"},
    # 세션 ID 기준으로 대화를 기록합니다.
    config={"configurable": {"session_id": "rag1234"}},
)

[대화 세션ID]: rag1234


'이 문서는 한 환자의 의료 기록을 요약한 것입니다. 이 환자는 77세 남성으로, 뇌내출혈과 수면장애 등의 진단을 받았으며, 폐렴과 고혈압 등의 합병증도 있습니다. 현재 항생제 치료를 받고 있으며, 경관식을 통해 영양을 공급받고 있습니다. 의료진은 환자의 상태를 지속적으로 모니터링하고 있으며, 환자의 상태를 개선하기 위해 노력하고 있습니다.'